In [1]:
import torch as t
import pandas as pd
import numpy as np
import plotly.express as px
import pytorch_lightning as pl
import wandb

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR
from fancy_einsum import einsum
from dataclasses import dataclass
from tqdm.notebook import tqdm_notebook
from einops import rearrange, reduce, repeat
from IPython.display import display
from typing import Callable
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import TQDMProgressBar

import sys 
sys.path.append('../common')

In [2]:
import general_modules as cm
import transformer_modules as tm
from transformer_modules import TransformerConfig
from nlp_modules import WordsDataset, WordsTokenizer


In [3]:
config = TransformerConfig(
    num_layers=12, 
    num_heads=8, 
    vocab_size=34543, 
    hidden_size=256,
    max_seq_len=16,
    dropout=0.1)

In [4]:
tokenizer = WordsTokenizer(config.max_seq_len)
words_ds = WordsDataset(seq_len=config.max_seq_len, filename='100-0.txt', tokenizer=tokenizer, truncate=0.01)
trainloader = DataLoader(words_ds, batch_size=16, shuffle=True)

In [5]:
class SHKTrainModule(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        # Key parameters
        self.save_hyperparameters(ignore=["model", "data"])
        self.model = tm.DecoderOnlyTransformer(config).train()

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        x = self.model(x)
        return x

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        logits = rearrange(logits, 'B S V -> (B S) V')
        y = rearrange(y, 'B S -> (B S)')
        loss = self.criterion(logits, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)

            # log prediction examples to wandb
            """
            pred = self.model(x)
            pred_keys = pred[0].sigmoid().tolist()
            pred_keys = [0 if p < self.hparams.thresh else 1 for p in pred_keys]


            mapper = cc.COCOCategorizer()
            pred_lbl = mapper.get_labels(pred_keys)
            
            try:
                self.logger.experiment.log({"val_pred_examples": [wandb.Image(x[0], caption=pred_lbl)]})
            except AttributeError:
                pass
            """

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        logits = rearrange(logits, 'B S V -> (B S) V')
        y = rearrange(y, 'B S -> (B S)')

        loss = self.criterion(logits, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = t.optim.AdamW(
            self.parameters()
        )

        return {"optimizer": optimizer}
        # return optimizer

In [6]:
pl_model = SHKTrainModule(config)

In [7]:
MODEL_FILENAME = "./w1d3_transformer_shakespeare.pt"

# Comment out if not using wandb
wandb_logger = WandbLogger(
    project="shk-transformer", 
    save_dir="training/logs/",
    log_model=True)
#wandb_logger.watch(pl_model, log="all")

trainer = pl.Trainer(
    max_epochs=1,
    precision=16,
    accelerator='cpu', 
    devices=1,
    logger=wandb_logger, # Comment out if not using wandb
    default_root_dir="training/checkpoints/",
    callbacks=[TQDMProgressBar(refresh_rate=10)])
trainer.fit(pl_model, train_dataloaders=trainloader)

#print(f"Saving model to: {MODEL_FILENAME}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: curt-tigges (arena-ldn). Use `wandb login --relogin` to force relogin
wandb: WARNING Path training/logs/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path training/logs/wandb/ wasn't writable, using system temp directory


/opt/conda/envs/shk/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:704: UserWarning: You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU. Using `precision='bf16'` instead.
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/shk/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name      | Type                   | Params
-----------------------------------------------------
0 | model     | DecoderOnlyTransformer | 18.3 M
1 | criterion | CrossEntropyLoss       | 0     
-----------------------------------------------------
18.3 M    Trainable params
0         Non-trainable pa

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [17]:
import sample_methods as s

model = t.load(MODEL_FILENAME, map_location=t.device('cpu'))
model.eval()

initial_text = "turn down for what"

text_output = s.sample_tokens(model, tokenizer, initial_text, max_tokens_generated=100, temperature=1.0, top_k=10)

print(text_output)

# turn down for what you do you think,
# That take the last, of many, which is so much I
# As this blows along than my life thou say’st, which makes thy hand,
# Thou wilt be given, or more
# Entitled in thy great world’s fresh blood will,
# To answer th’ alluring countenance, beauty

turn down for what the world that should not that which that it that which it the he day,
  In that which it, nor that it love that your sweet love?
Why with more love for me, let thy love’s love shall be,
  To thee in my love that me thou art.
When
